In [1]:
#import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import os
import numpy as np
from scipy.stats import sem

In [6]:
#set variables for data files
mouse_metadata = os.path.join("Resources", "mouse_metadata.csv")
study_files = os.path.join("Resources", "study_results.csv")

#read the csv files
mouse_metadata = pd.read_csv(mouse_metadata)
study_files = pd.read_csv(study_files)
mouse_metadata.head()

#combine the data sets into one & preview
data_df = pd.merge(mouse_metadata, study_files, on="Mouse ID")
data_df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [8]:
#Summary Statistics
#find: mean, median, variance, standard deviation, and standard error of measurement

#group dataset by Drug Regimen and Timepoint, then find stats on Tumor Volume
tumor_mean = data_df.groupby(["Drug Regimen", "Timepoint"]).mean()["Tumor Volume (mm3)"]
tumor_median = data_df.groupby(["Drug Regimen", "Timepoint"]).median()["Tumor Volume (mm3)"]
tumor_variance = data_df.groupby(["Drug Regimen", "Timepoint"]).var()["Tumor Volume (mm3)"]
tumor_std = data_df.groupby(["Drug Regimen", "Timepoint"]).std()["Tumor Volume (mm3)"]
tumor_sem = data_df.groupby(["Drug Regimen", "Timepoint"]).sem()["Tumor Volume (mm3)"]

#put the stats into a dataframe
tumor_vol_stats_df = pd.DataFrame({"Mean": tumor_mean,
                                 "Median": tumor_median,
                                 "Variance": tumor_variance,
                                 "Standard Deviant": tumor_std,
                                 "Standard Error": tumor_sem})

#reset index & preview
tumor_vol_stats_df = tumor_vol_stats_df.reset_index()
tumor_vol_stats_df

,Drug Regimen,Timepoint,Mean,Median,Variance,Standard Deviant,Standard Error
0,Capomulin,0,45.000000,45.000000,0.000000,0.000000,0.000000
1,Capomulin,5,44.266086,45.597064,5.030889,2.242964,0.448593
2,Capomulin,10,43.084291,43.421014,12.344133,3.513422,0.702684
3,Capomulin,15,42.064317,42.798160,16.878693,4.108369,0.838617
4,Capomulin,20,40.716325,40.716428,19.035028,4.362915,0.909731
...,...,...,...,...,...,...,...
95,Zoniferol,25,55.432935,55.676604,5.808348,2.410052,0.602513
96,Zoniferol,30,57.713531,57.419744,9.601024,3.098552,0.800043
97,Zoniferol,35,60.089372,60.365832,10.876760,3.297993,0.881426
98,Zoniferol,40,62.916692,62.274543,13.958456,3.736102,0.998515


In [ ]:
#testing the counts and various numbers so I can check against my functions below
#keep for now, delete for final output

drug_types = data_df["Drug Regimen"].unique()
print(drug_types)

drug_counts = data_df["Drug Regimen"].value_counts()
print(drug_counts)

In [ ]:
#Summary Statistics
#generate a summary statistics table of mean, median, variance, 
#standard deviation, and SEM of the tumor volume for each regimen

#generate stats
tumor_mean = data_df.groupby("Drug Regimen").mean()["Tumor Volume (mm3)"]
tumor_median = data_df.groupby("Drug Regimen").median()["Tumor Volume (mm3)"]
tumor_variance = data_df.groupby("Drug Regimen").var()["Tumor Volume (mm3)"]
tumor_std = data_df.groupby("Drug Regimen").std()["Tumor Volume (mm3)"]
tumor_sem = data_df.groupby("Drug Regimen").sem()["Tumor Volume (mm3)"]

#place into a summary table
tumor_vol_stats_df = pd.DataFrame({"Mean": tumor_mean,
                                 "Median": tumor_median,
                                 "Variance": tumor_variance,
                                 "Standard Deviant": tumor_std,
                                 "Standard Error": tumor_sem})
tumor_vol_stats_df

In [ ]:
#use Pandas to create a bar plot with the above data
multi_stats = tumor_vol_stats_df.plot(kind="bar", figsize=(10,5))

plt.show()

In [ ]:
#use Matplotlib to create a bar plot with the same data







In [ ]:
#Create pie charts (one using Pandas, the other Matplotlib) to show the mice gender distribution in the study

#the total number of mice
total_mice = len(data_df["Mouse ID"].unique())
#delete print command
print(total_mice)

#removing duplicate "Mouse ID" rows
gender_df = data_df.loc[:, ["Mouse ID", "Sex"]]
gender_df = gender_df.drop_duplicates()

#find the total of males and females
gender_total = gender_df["Sex"].value_counts()
#delete print and type commands
print(gender_total)
type(gender_total)

gender_df = pd.DataFrame({"Gender": gender_total})
#delete below
gender_df


In [ ]:
#generate a pie plot showing the distribution of female versus male mice using pandas
gender_pie = gender_df.plot.pie(y="Gender", figsize=(5,5))

In [ ]:
#generate a pie plot showing the distribution of female versus male mice using matplotlib
labels = ["Male", "Female"]
colors = ["red", "orange"]
sizes = gender_total
#explode = (0.1, 0)
plt.pie(sizes, labels=labels, colors=colors, autopct="%1.1f%%", shadow=True, startangle=140)

In [ ]:
#calculate the final tumor volume of each mouse across four of the most promising treatment regimens
#calculate the IQR and quantitatively determine if there are any potential outliers

#first group my data by Timepoint and Drug Regimen
#then look at Tumor Volume
grouped_df = data_df.groupby(["Drug Regimen", "Timepoint"])
MeanTumorVolume = grouped_df["Tumor Volume (mm3)"].mean()

#create dataframe from the series created above
MeanTumorVolume_df = pd.DataFrame(MeanTumorVolume)

#need Drug Regimen & Timepoint as columns, so reset index
MeanTumorVolume_df = MeanTumorVolume_df.reset_index()
MeanTumorVolume_df

In [ ]:
#reformat in order to find four 'best' drug treatments and for future plotting (?????? on the future plotting)
pivot_MeanTumorVolume_df = MeanTumorVolume_df.pivot(index="Timepoint", columns="Drug Regimen", values="Tumor Volume (mm3)")
pivot_MeanTumorVolume_df

#4 'best' treatments: Ramicane, Capomulin, Ceftamin, Infubinol

In [ ]:
#calculate the final tumor volume of each mouse across four of the most promising treatment regimens
top_four = pivot_MeanTumorVolume_df[["Ramicane", "Capomulin", "Ceftamin", "Infubinol"]]
top_four
#calculate the IQR and quantitatively determine if there are any potential outliers. 

In [ ]:
ramicane_result = top_four.iloc[9, 0]
capomulin_result = top_four.iloc[9, 1]
ceftamin_result = top_four.iloc[9, 2]
infubinol_result = top_four.iloc[9, 3]

results_df = pd.DataFrame({"Ramicane": [ramicane_result],
                           "Capomulin": [capomulin_result],
                           "Ceftamin": [ceftamin_result],
                           "Infubinol": [infubinol_result]})
results_df

In [ ]:
MeanTumorVolume_df = MeanTumorVolume_df.rename(columns={"Drug Regimen": "Drug"})
MeanTumorVolume_df

drugs = ["Ramicane", "Capomulin", "Ceftamin", "Infubinol"]
MeanTumorVolume_df.Drug.isin(drugs)
top_drugs = MeanTumorVolume_df[MeanTumorVolume_df.Drug.isin(drugs)]
top_drugs

In [ ]:
end_time = top_drugs["Timepoint"] == 45
final_tumor = top_drugs[end_time]
final_tumor

In [ ]:
#THE FINAL TUMOR SIZE FOR THE TOP FOUR DRUGS (only four values)
volumes = final_tumor["Tumor Volume (mm3)"]

fig1, ax1 = plt.subplots()
ax1.set_title('Tumors')
ax1.set_ylabel('Tumor Volume')
ax1.boxplot(volumes)
plt.show()

In [ ]:
#THE FINAL TUMOR SIZE FOR THE TOP FOUR DRUGS (only four values)
quartiles = volumes.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of tumor volumes is: {lowerq}")
print(f"The upper quartile of tumor volumes is: {upperq}")
print(f"The interquartile range of tumor volumes is: {iqr}")
print(f"The the median of tumor volumes is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

In [ ]:
#ALL TUMOR VOLUMES, ALL DRUGS
volumes = data_df["Tumor Volume (mm3)"]

fig1, ax1 = plt.subplots()
ax1.set_title('Tumors')
ax1.set_ylabel('Tumor Volume')
ax1.boxplot(volumes)
plt.show()

In [ ]:
#ALL TUMOR VOLUMES, ALL DRUGS
quartiles = volumes.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of tumor volumes is: {lowerq}")
print(f"The upper quartile of tumor volumes is: {upperq}")
print(f"The interquartile range of tumor volumes is: {iqr}")
print(f"The the median of tumor volumes is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

In [ ]:
#ALL TUMOR VOLUMES OF THE TOP FOUR DRUGS
volumes = top_drugs["Tumor Volume (mm3)"]

fig1, ax1 = plt.subplots()
ax1.set_title('Tumors')
ax1.set_ylabel('Tumor Volume')
ax1.boxplot(volumes)
plt.show()

In [ ]:
#ALL TUMOR VOLUMES OF THE TOP FOUR DRUGS
quartiles = volumes.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of tumor volumes is: {lowerq}")
print(f"The upper quartile of tumor volumes is: {upperq}")
print(f"The interquartile range of tumor volumes is: {iqr}")
print(f"The the median of tumor volumes is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")